<a href="https://colab.research.google.com/github/AUT-Student/NN-HW3/blob/main/NN_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras

# Dataset

In [7]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip -O "/content/Dataset.zip"

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
To: /content/Dataset.zip
100% 61.0M/61.0M [00:03<00:00, 17.8MB/s]


In [10]:
!unzip /content/Dataset.zip

Archive:  /content/Dataset.zip
replace UCI HAR Dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [11]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names -O "/content/Dataset.names"

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names
To: /content/Dataset.names
100% 6.30k/6.30k [00:00<00:00, 11.3MB/s]


In [12]:
!cat "/content/Dataset.names"

Human Activity Recognition Using Smartphones Dataset
Version 1.0
Jorge L. Reyes-Ortiz(1,2), Davide Anguita(1), Alessandro Ghio(1), Luca Oneto(1) and Xavier Parra(2)
1 - Smartlab - Non-Linear Complex Systems Laboratory
DITEN - Universit�  degli Studi di Genova, Genoa (I-16145), Italy. 
2 - CETpD - Technical Research Centre for Dependency Care and Autonomous Living
Universitat Polit�cnica de Catalunya (BarcelonaTech). Vilanova i la Geltr� (08800), Spain
activityrecognition '@' smartlab.ws 

The experiments have been carried out with a group of 30 volunteers within an age bracket of 19-48 years. Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (Samsung Galaxy S II) on the waist. Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity at a constant rate of 50Hz. The experiments have been video-recorded to label the data manually. The obtained data

In [13]:
!cat "/content/UCI HAR Dataset/activity_labels.txt"

1 WALKING
2 WALKING_UPSTAIRS
3 WALKING_DOWNSTAIRS
4 SITTING
5 STANDING
6 LAYING


In [14]:
X_train = pd.read_csv("/content/UCI HAR Dataset/train/X_train.txt", delimiter=r"\s+", header=None)
X_test = pd.read_csv("/content/UCI HAR Dataset/test/X_test.txt", delimiter=r"\s+", header=None)
y_train = pd.read_csv("/content/UCI HAR Dataset/train/y_train.txt", delimiter=r"\s+", header=None)
y_test = pd.read_csv("/content/UCI HAR Dataset/test/y_test.txt", delimiter=r"\s+", header=None)

In [15]:
X_train["Class"] = y_train
X_test["Class"] = y_test

In [16]:
X_valid, X_train = np.split(X_train.sample(frac=1, random_state=0), [int(.125*len(X_train))])

In [17]:
y_train = X_train["Class"]
X_train = X_train.drop(columns=["Class"])

y_valid = X_valid["Class"]
X_valid = X_valid.drop(columns=["Class"])

In [18]:
y_train = y_train.apply(lambda x:x-1)
y_test = y_test.apply(lambda x:x-1)
y_valid = y_valid.apply(lambda x:x-1)

# MLP Classifer

In [79]:
class MLPModel(keras.Model):
  def __init__(self, number_hidden_layers, number_hidden_units):
    super().__init__()

    if number_hidden_layers<0:
      raise Exception("The number of hidden layers must be a non-negetive number")

    if number_hidden_layers != len(number_hidden_units):
      raise Exception("The number of hidden layers must equal to the length of the number of hidden units list")

    self.model = keras.models.Sequential()

    for i in range(number_hidden_layers):
      self.model.add(keras.layers.Dense(units=number_hidden_units[i], activation="relu", name=f"Dense_Layer_{i+1}"))

    self.model.add(keras.layers.Dense(6, activation="softmax", name="Output_Layer"))

  def call(self, inputs):
    return self.model(inputs)

In [123]:
def train_and_evaluate(number_hidden_layers, number_hidden_units, learning_rate):
  es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", restore_best_weights=True, patience=5)

  model = MLPModel(number_hidden_layers=number_hidden_layers, number_hidden_units=number_hidden_units)

  opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  
  model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  model.fit(x=X_train, y=y_train, epochs=30, validation_data=(X_valid, y_valid), verbose=0, callbacks=[es_callback])

  train_accuracy = model.evaluate(X_train, y_train, verbose=0)[1]
  validation_accuracy = model.evaluate(X_valid, y_valid, verbose=0)[1]

  print(f"Train = {round(train_accuracy*100, 2)}, Validation = {round(validation_accuracy*100, 2)}")

# Learning Rate Fine-tuning

In [124]:
for learning_rate in [0.0001, 0.0005, 0.001, 0.003, 0.005, 0.01, 0.1]:
  print(f"Learning Rate = {learning_rate}")
  train_and_evaluate(number_hidden_layers=3, number_hidden_units=[128, 64, 32], learning_rate=0.001)
  print("")

Learning Rate = 0.0001
Train = 98.83, Validation = 97.71

Learning Rate = 0.0005
Train = 98.85, Validation = 97.82

Learning Rate = 0.001
Train = 97.95, Validation = 96.52

Learning Rate = 0.003
Train = 98.8, Validation = 98.04

Learning Rate = 0.005
Train = 98.4, Validation = 97.17

Learning Rate = 0.01
Train = 98.77, Validation = 97.93

Learning Rate = 0.1
Train = 98.57, Validation = 97.93



# Number Layer and Number Neuron Fine-tuning

In [125]:
train_and_evaluate(number_hidden_layers=0, number_hidden_units=[], learning_rate=0.003)
print("")

Train = 98.82, Validation = 97.93



In [126]:
for number_hidden_units in [[32], [64], [128]]:
  print(f"number_hidden_units = {number_hidden_units}")
  train_and_evaluate(number_hidden_layers=1, number_hidden_units=number_hidden_units, learning_rate=0.003)
  print("")

number_hidden_units = [32]
Train = 98.96, Validation = 97.93

number_hidden_units = [64]
Train = 98.87, Validation = 97.82

number_hidden_units = [128]
Train = 98.87, Validation = 98.04



In [127]:
for number_hidden_units in [
                            [32, 32], [64, 32], [128, 128]
                            ]:
  print(f"number_hidden_units = {number_hidden_units}")
  train_and_evaluate(number_hidden_layers=2, number_hidden_units=number_hidden_units, learning_rate=0.003)
  print("")

number_hidden_units = [32, 32]
Train = 98.87, Validation = 98.26

number_hidden_units = [64, 32]
Train = 98.4, Validation = 97.06

number_hidden_units = [128, 128]
Train = 97.28, Validation = 96.63



In [128]:
for number_hidden_units in [
                            [64, 32, 32], [64, 64, 64], [128, 64, 32]
                            ]:
  print(f"number_hidden_units = {number_hidden_units}")
  train_and_evaluate(number_hidden_layers=3, number_hidden_units=number_hidden_units, learning_rate=0.003)
  print("")

number_hidden_units = [64, 32, 32]
Train = 98.49, Validation = 97.71

number_hidden_units = [64, 64, 64]
Train = 98.15, Validation = 97.39

number_hidden_units = [128, 64, 32]
Train = 98.26, Validation = 97.28



In [129]:
for number_hidden_units in [
                            [128, 128, 64, 64], [128, 64, 32, 16], [128, 64, 32, 64]
                            ]:
  print(f"number_hidden_units = {number_hidden_units}")
  train_and_evaluate(number_hidden_layers=4, number_hidden_units=number_hidden_units, learning_rate=0.05)
  print("")

number_hidden_units = [128, 128, 64, 64]
Train = 18.76, Validation = 18.17

number_hidden_units = [128, 64, 32, 16]
Train = 52.45, Validation = 50.82

number_hidden_units = [128, 64, 32, 64]
Train = 18.76, Validation = 18.17



# SOM

In [93]:
class SOM():
  def __init__(self, number_epochs, map_size, number_features,
               initial_learning_rate, initial_neighbor_strength,
               raduis_neighbor,
               learning_rate_decay, neighbor_strength_decay):
    
    self.number_epochs = number_epochs
    self.map_size = map_size
    self.number_features = number_features
    self.learning_rate = initial_learning_rate
    self.neighbor_strength = initial_neighbor_strength
    self.raduis_neighbor = raduis_neighbor
    self.learning_rate_decay = learning_rate_decay
    self.neighbor_strength_decay = neighbor_strength_decay
    self.weight = np.random.random((self.map_size[0], self.map_size[1], self.number_features))

  def train(self, X_train):
    for epoch in range(self.number_epochs):
      print(f"epoch = {epoch}")
      for i, row in enumerate(X_train):
        NS = self.get_NS(row)
        self.update_weights(NS, row)

      self.learning_rate *= self.learning_rate_decay
      self.neighbor_strength *= self.neighbor_strength_decay

  def get_NS(self, row):
    winner_distance = np.inf
    winner = (None, None)

    for i in range(self.map_size[0]):
      for j in range(self.map_size[1]):
        neuron_weight = self.weight[i][j]
        neuron_distance = self.distance(neuron_weight, row)
        
        if neuron_distance < winner_distance:
          winner_distance = neuron_distance
          winner = (i, j)
    
    winner_weight = self.weight[winner]

    NS = np.zeros(self.map_size)

    for i in range(self.map_size[0]):
      for j in range(self.map_size[1]):
        neuron_weight = self.weight[i][j]
        neuron_distance = self.distance(neuron_weight, winner_weight)
        if neuron_distance < self.raduis_neighbor:
          NS[i][j] = 1-neuron_distance/self.raduis_neighbor
    
    return NS

  def update_weights(self, NS, row):
    for i in range(self.map_size[0]):
      for j in range(self.map_size[1]):
        self.weight[i][j] += NS[i][j] * self.learning_rate * self.distance(row, self.weight[i][j])

  def visualize(self):
    pass

  @staticmethod
  def distance(array1, array2):
    return np.linalg.norm(array1 - array2)

In [94]:
som = SOM(number_epochs=10, map_size=(9, 9), number_features=len(X_train.columns),
               initial_learning_rate=0.0001, initial_neighbor_strength=0.1,
               raduis_neighbor=20.0,
               learning_rate_decay=0.9, neighbor_strength_decay=0.9)

In [95]:
som.train(X_train.to_numpy())

epoch = 0
epoch = 1
epoch = 2
epoch = 3
epoch = 4
epoch = 5
epoch = 6
epoch = 7
epoch = 8
epoch = 9


In [96]:
som.weight

array([[[50752.58745165, 50753.36608935, 50752.65081598, ...,
         50752.75434189, 50752.73495598, 50752.95445508],
        [50739.15970881, 50739.05602149, 50739.46190114, ...,
         50739.53520669, 50738.82992903, 50738.96856949],
        [50760.7241821 , 50760.81319263, 50760.16481522, ...,
         50760.57230655, 50760.1152562 , 50760.03199663],
        ...,
        [50747.62561719, 50747.87427408, 50747.75024272, ...,
         50747.88427086, 50747.49804552, 50748.30835376],
        [50761.63004684, 50762.02033929, 50762.48742447, ...,
         50762.08101565, 50761.63734287, 50761.91101642],
        [50743.73932585, 50742.96151056, 50743.00237789, ...,
         50742.86767201, 50743.73167002, 50742.77513016]],

       [[50731.89432477, 50731.92654993, 50731.77435665, ...,
         50731.44874876, 50731.24842422, 50731.25198817],
        [50763.40401792, 50763.50065667, 50763.95737084, ...,
         50763.53955972, 50763.05186218, 50763.35383503],
        [50732.61100442, 